<div>
<table style="width: 100%">
	<tr>
		<td>
		<table style="width: 100%">
			<tr>
                <td ><center><font size="5"><b>Module 52</b></font><center>
                <center><font size="6">Wetlands for Livelihood & Conservation</font><center></td>
			</tr>
			<tr>
                <td><center><font size="14">Notebook</font><center></td>
			</tr>
			<tr>
                <td><center><font size="6"><b>Wetland change analysis in Google Earth Engine (GEE)</b></font><center></td>
			</tr>
		</table>
		</td> 
		<td><center><img src='https://github.com/spareeth/module6_wetland/blob/master/images/ihe-delft-institute_unesco_fc-lr.jpg?raw=true'></img></td>
	</tr>
</table>
</div>

# Table of contents
1. [Learning objectives](#learningobs)
2. [Introduction](#introduction)
3. [Installation of required library](#install)
4. [Initializing the GEE](#initial)
5. [Import data in GEE](#import)
6. [Import Global LULC in GEE](#importglad)
7. [Define study area & Clip](#clip)
8. [Compute change area statistics](#area)
9. [Image collection and temporal aggregation](#collection)
10. [Exercise](#exercise)
11. [References and tutorials](#reference)

# 1. Learning objectives<a name="learningobs"></a>

- To access data from Google Earth Engine (GEE) using Python
- To import global spatial data and interactively visualise
- Interactively define a study area
- Assess land use land cover dynamics globally and over study area
- Analyse the land land use cover CHANGE dynamics over the study area for twenty years (2000-2020)
- Analyse the long term precipitation data for forty years

# 2. Introduction<a name="introduction"></a>

[Google Earth Engine (GEE)](https://earthengine.google.com/) is a computing platform that allows users to run geospatial analysis on Google's infrastructure. It brings together the world's satellite imagery from multiple sources – Landsat, Sentinel, MODIS etc. along with access to other secondary climatic and Geo-physical datasets covering entire globe. GEE has become very popular in the geospatial community and numerous environmental applications at local, regional, and global scales have been deployed all over the globe. GEE provides both JavaScript and Python APIs for making computational requests to the Earth Engine servers. 

Watch [this video](https://youtu.be/gKGOeTFHnKY) for an overview about GEE and what it can do.

In this module we will use [Geemap](https://geemap.org/), which is a Python package for interactive mapping with Google Earth Engine (GEE). It is built upon ipyleaflet and ipywidgets, and enables users to analyze and visualize Earth Engine datasets interactively within a Jupyter-based environment.

Read below paper about Geemap by the developed Prof. Quisheng Wu. <br>
* Wu, Q., (2020). geemap: A Python package for interactive mapping with Google Earth Engine. The Journal of Open Source Software, 5(51), 2305. https://doi.org/10.21105/joss.02305
    
<b>Note</b>: You will need a valid login to access GEE. See [this document](docs/Signup_Instructions_GEE.pdf) for instructions to signup.

# 3. Installation of required library<a name="install"></a>
You need to have geemap python library installed in your computer. <br>
If you are using conda environment then use the below command to install geemap. <br>

In [ ]:
#conda install -c conda-forge geemap

<b>OR</b> use pip to install the geemap package as given below

In [ ]:
#!pip install --upgrade geemap

In [ ]:
#geemap.update_package()

# 4. Import Libraries & Initializing the GEE<a name="initial"></a>
Let us start with geemap by importing the library and bringing in a interactive map.

In [ ]:
import os
import ee
import geemap
import pandas as pd

Now you will define a interactive map and get it inline in this notebook. <br>
<b> Note that first time you do this step in your computer, it will ask for authorize the access needed by GEE. You have to follow the instructions and finish the authorization. This is only one time in local computer</b> <br>
See this [youtube video](https://www.youtube.com/watch?v=h0pz3S6Tvx0&t=359s) showing the authorization steps.

In [ ]:
Map = geemap.Map()
Map

Now that you have successfully initiated the geemap, let us explore data sets from GEE by importing them to this notebook.

# 5. Import data in GEE<a name="import"></a>
GEE offers access to petabytes of data from various data sources primarily from satellites and remote sensing.<br>
You can check [this link](https://developers.google.com/earth-engine/datasets) for the data catalog of GEE.<br>
<br>
For each type of data, the catalog gives you the metadata and <b>"Earthengine snippet"</b> as shown below (only an example) which can be used to access this particular data. Below shown is the HydroSHEDS Digital Elevation Model (DEM) at 90m.<br>
<br>
<img align="left" src="https://github.com/spareeth/module6_wetland/blob/master/images/gee1.png?raw=true" alt="gee" width="800"/> <br>
<br>

**Let us now add the DEM to the interactive map above**

Let us import the HydroSHEDS Digital Elevation Model (DEM) at 90m and display it in the Map above. See details of [HydroSHEDS Digital Elevation Model (DEM)](https://developers.google.com/earth-engine/datasets/catalog/WWF_HydroSHEDS_03CONDEM). Find more details abour HydroSHEDS [here](https://www.hydrosheds.org/)

**GEE Functions used** \
[*ee.Image*](https://developers.google.com/earth-engine/apidocs/ee-image): An object to represent an Earth Engine image. \
[*Map.addLayer*](https://geemap.org/geemap/#geemap.geemap.Map.addLayer): Adds a given EE object to the map as a layer.

In [ ]:
dem = ee.Image("WWF/HydroSHEDS/03CONDEM").select('b1')
elevParams = {min: -20, max: 3000, 'gamma': 0.5};
Map.addLayer(dem, elevParams, "dem")

Let us import the global basin boundaries (vector) HydroSHEDS in GEE and display it in the Map above. See details of [Hydrobasins Level 3](https://developers.google.com/earth-engine/datasets/catalog/WWF_HydroSHEDS_v1_Basins_hybas_3).

**GEE Functions used** \
[*ee.FeatureCollection*](https://developers.google.com/earth-engine/apidocs/ee-featurecollection): An object to represent an Earth Engine collection of vector, geometry etc. \
[*Map.addLayer*](https://geemap.org/geemap/#geemap.geemap.Map.addLayer): Adds a given EE object to the map as a layer.

In [ ]:
basins = ee.FeatureCollection('WWF/HydroSHEDS/v1/Basins/hybas_3');
visualization = {'color': '808080', 'strokeWidth': 1}
Map.addLayer(basins, visualization, "Hydro Basins")

# 6. Import Global LULC in GEE<a name="importglad"></a>
In this tutorial the newly published global land use land cover data by researchers in University of Maryland will be used.<br> The global 30-m spatial resolution dataset quantifies changes in forest extent and height, cropland, built-up lands, surface water, wetlands and perennial snow and ice extent from the **year 2000 to 2020**. Landsat Analysis Ready Data served as an input for land cover and use mapping. Each thematic product was independently derived using locally and regionally calibrated machine learning tools (see references below). <br>
<br>
More details about this dataset is provided [here](https://glad.umd.edu/dataset/GLCLUC2020).<br>
Read below paper for more details on this dataset. <br>
* P.V. Potapov, M.C. Hansen, A.H. Pickens, A. Hernandez-Serna, A. Tyukavina, S. Turubanova, V. Zalles, X. Li, A. Khan, F. Stolle, N. Harris, X.-P. Song, A. Baggett, I. Kommareddy, A. Komareddy (2022). https://doi.org/10.3389/frsen.2022.856903

<br>
A high resolution change map is also produced showing the global changes in these classes over twenty years.<br>

User can also explore this data in a GEE based application which can be accessed [here](https://glad.earthengine.app/view/glcluc-2000-2020).<br>
<br>
**Below image is the Legend to read the LULC maps (2000 and 2020) and the change map.**
<br>
<img align="left" src="https://github.com/spareeth/module6_wetland/blob/master/images/glad_legend.png?raw=true" alt="gee" width="500"/> <br>
<br>

<b>Note</b>: Here is the document (excel) with [detailed legend](docs/GLAD_legend.xlsx).<br>
<br>
Now let us start a new Map instance and import the LULC maps.

In [ ]:
Map = geemap.Map()
Map

In [ ]:
landmask = ee.Image("projects/glad/landBuffer4").mask();
change =ee.Image('projects/glad/GLCLU2020/LCLUC').updateMask(landmask);
lulc00 = ee.Image('projects/glad/GLCLU2020/LCLUC_2000').updateMask(landmask)
lulc20 = ee.Image('projects/glad/GLCLU2020/LCLUC_2020').updateMask(landmask);
change =ee.Image('projects/glad/GLCLU2020/LCLUC').updateMask(landmask);

visParamMap = {"min":0,"max":255,"palette":["FEFECC","FAFAC3","F7F7BB","F4F4B3","F1F1AB","EDEDA2","EAEA9A","E7E792","E4E48A","E0E081","DDDD79","DADA71","D7D769","D3D360","D0D058","CDCD50","CACA48","C6C63F","C3C337","C0C02F","BDBD27","B9B91E","B6B616","B3B30E","B0B006","609C60","5C985C","589558","549254","508E50","4C8B4C","488848","448544","408140","3C7E3C","387B38","347834","317431","2D712D","296E29","256B25","216721","1D641D","196119","155E15","115A11","0D570D","095409","065106","643700","643a00","643d00","644000","644300","644600","644900","654c00","654f00","655200","655500","655800","655a00","655d00","656000","656300","666600","666900","666c00","666f00","667200","667500","667800","667b00","ff99ff","FC92FC","F98BF9","F685F6","F37EF3","F077F0","ED71ED","EA6AEA","E763E7","E45DE4","E156E1","DE4FDE","DB49DB","D842D8","D53BD5","D235D2","CF2ECF","CC27CC","C921C9","C61AC6","C313C3","C00DC0","BD06BD","bb00bb","000003","000004","000005","BFC0C0","B7BDC2","AFBBC4","A8B8C6","A0B6C9","99B3CB","91B1CD","89AFD0","82ACD2","7AAAD4","73A7D6","6BA5D9","64A3DB","5CA0DD","549EE0","4D9BE2","4599E4","3E96E6","3694E9","2E92EB","278FED","1F8DF0","188AF2","1088F4","0986F7","55A5A5","53A1A2","519E9F","4F9B9C","4D989A","4B9597","499294","478F91","458B8F","43888C","418589","3F8286","3D7F84","3B7C81","39797E","37767B","357279","336F76","316C73","2F6970","2D666E","2B636B","296068","285D66","bb93b0","B78FAC","B48CA9","B189A6","AE85A2","AA829F","A77F9C","A47B99","A17895","9E7592","9A718F","976E8C","946B88","916885","8D6482","8A617F","875E7B","845A78","815775","7D5472","7A506E","774D6B","744A68","714765","de7cbb","DA77B7","D772B3","D46EAF","D169AB","CE64A8","CB60A4","C85BA0","C4579C","C15298","BE4D95","BB4991","B8448D","B54089","B23B86","AF3682","AB327E","A82D7A","A52976","A22473","9F1F6F","9C1B6B","991667","961264","000000","000000","000000","1964EB","1555E4","1147DD","0E39D6","0A2ACF","071CC8","030EC1","0000BA","0000BA","040464","0000FF","3051cf","000000","000000","000000","000000","000000","000000","000000","000000","000000","000000","000000","000000","000000","000000","000000","000000","000000","000000","000000","000000","547FC4","4D77BA","466FB1","4067A7","395F9E","335895","335896","335897","ff2828","ffffff","d0ffff","ffe0d0","ff7d00","fac800","c86400","fff000","afcd96","afcd96","64dcdc","00ffff","00ffff","00ffff","111133","000000"]};
Map.addLayer(lulc00, visParamMap, "Land Cover 2000")
Map.addLayer(lulc20, visParamMap, "Land Cover 2020")
Map.addLayer(change, visParamMap, "Land Cover Change 2000 - 2020")

**Let us have a detailed look into the change map over a high resolution basemap and split window**

In [ ]:
left_layer = geemap.ee_tile_layer(change, visParamMap, name='Land Cover Change 2000 - 2020')

In [ ]:
Map = geemap.Map()
Map.split_map(left_layer, 'HYBRID', left_label='Basemap', right_label='Change 2000 - 2020',)
Map

# 7. Define study area & Clip<a name="clip"></a>
GEE offers 'algorithms' to conduct spatial data analysis. The analyses can be performed on the data offered in GEE and also with the user imported data.
Here we are clipping the global landcover map to a user defined study area (interactively in map)

**Functions used** \
[*clip*](https://developers.google.com/earth-engine/apidocs/ee-featurecollection-filter): Clips an image to a Geometry or Feature. \
[*Map.addLayer*](https://geemap.org/geemap/#geemap.geemap.Map.addLayer):Adds a given EE object to the map as a layer.

**Let us define our study area/region of interest(roi) for further analysis** <br>
<br>
In the below map, **draw your roi using the drawing tools provided on the left side.**

In [ ]:
Map = geemap.Map()
Map

In [ ]:
Map.draw_features

In [ ]:
roi=ee.FeatureCollection(Map.draw_features)
change_roi = change.clip(roi)
Map.addLayer(change_roi, visParamMap, 'Change 2020-2020 ROI')

# 8. Compute change area statistics<a name="area"></a>
In this section, let us compute the change area statistics in Sq.Km

**Functions used** \
[*image_area_by_group*](https://geemap.org/common/#geemap.common.image_area_by_group): Calculates the area of each class of an image.

In [ ]:
# Compute area of each land cover type in Sq.Km
df = geemap.image_area_by_group(
    change_roi, groups=None, scale=1000, denominator=1e6, decimal_places=4, verbose=True
)
df

**Save the above table to a csv (excel) for further analysis**

In [ ]:
df['class'] = df.index
df.to_csv('change_area.csv', index=False)

**As we are mainly interested in the major change classes, let us filter all the classes below 240. Refer to the legend.xlsx file**

In [ ]:
df1 = df.loc[df['class'].astype(int) >= 240]

**Plot the change area as a bar plot.**

In [ ]:
df1.plot.bar(y='area', x='class')

<br>
<img align="left" src="https://github.com/spareeth/module6_wetland/blob/master/images/change_legend.png?raw=true" alt="gee" width="400"/>
<br>

<h2><b><span style="color:Green">Question to Explore!</span></b></h2>

How will you create a bar plot for Land cover area in 2000 and 2020 separately?

# 9. Image collection and temporal aggregation <a name="collection"></a>
In GEE, a stack or time series of images are called Image Collections. GEE offers all time series data related to satellite data, climate, land cover etc as image collections. An ImageCollection can be loaded by pasting an Earth Engine asset ID into the ImageCollection constructor. You can find ImageCollection IDs in the [data catalog](https://developers.google.com/earth-engine/datasets). 
For example, to load the GPM Precipitation data (Rainfall), go to the [product page](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_MONTHLY) in data catalog and get the asset id given under *Earth Engine Snippet* as shown below.
<br>
<img src="https://github.com/spareeth/module6_wetland/blob/master/images/gee2.png?raw=true" align="left" alt="gee" width="800"/>
<br>

**Functions used** \
[*ee.ImageCollection*](https://geemap.org/common/#geemap.common.image_area_by_group): Calculates the area of each class of an image. \
[*filterBounds*](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-filterbounds): Shortcut to filter a collection by intersection with geometry. Items in the collection with a footprint that fails to intersect the given geometry will be excluded. \
[*filterDate*](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-filterdate): Shortcut to filter a collection by a date range. \
[select](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-select): Select bands from each image in a collection.
<br>
<br>
**Next, let us bring in monthly ERA5 total precipitation data and filter it to your study area for all the years from 1979 to 2019.** \
**note that the unit is m, but later converted to mm while plotting**

In [ ]:
pcp_monthly = ee.ImageCollection("ECMWF/ERA5/MONTHLY") \
    .filterBounds(roi) \
    .filterDate('1979-01-01', '2020-01-01') \
    .select('total_precipitation')

Let us now compute the univariate statistics (mean) of the annual precipitation maps over your roi.
<br>
**Functions used** \
[*image_mean_value*](https://geemap.org/common/#geemap.common.image_mean_value):Retrieves the mean value of an image.\
[*getInfo*](https://developers.google.com/earth-engine/apidocs/ee-string-getinfo):  Retrieves the value of this object from the server.

In [ ]:
# Function to get annual precip image from monthly precip images per year
def annual(year):
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = start_date.advance(1, 'year')
    
    image = pcp_monthly.select('total_precipitation') \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .sum()
    stat = geemap.image_mean_value(image, roi, scale=1000)
    return stat

In [ ]:
# Apply the above function to compute annual Precipitation maps
years = ee.List.sequence(1979, 2019)
annualimages = years.map(annual).getInfo()
years_list = years.getInfo()

In [ ]:
df2 = pd.DataFrame(annualimages, columns = ['total_precipitation'])
df2['year'] = years_list
df2['precip_mm'] = df2['total_precipitation'] * 1000

In [ ]:
df2.plot.bar(y='precip_mm', x='year', figsize=(12,5))

**Save the above table to a csv (excel) file for further analysis**

In [ ]:
df2.to_csv('Annual_precip_ROI.csv', index=False)

In the final step download the clipped change map to your google drive in geotif format for further analysis in other GIS softwares like QGIS. \
**Note that depending on the area of interest export can take long time to finish**

In [ ]:
geemap.ee_export_image_to_drive(
    change_roi, region=roi.geometry(), description='lulc_change', fileNamePrefix='change', folder='gee_output'
)

# 10. Exercise <a name="exercise"></a>

Now that you have learned how to perform change analysis in GEE focusing on wetlands, repeat the same steps (in a new notebook) for any wetland from this list below:

* Lake Naivasha, Kenya 
* Nyando wetland, Kenya 
* Namatala wetland, Uganda 
* Nakivubo wetland, Uganda 
* Lubigi wetland, Uganda 
* Kafue flats, Zambia
* East Kolkata Wetlands, India
* Bhoj Wetlands, India
* Chilka Lake, India 

**Your Tasks**

* Prepare a map showing the changes in your wetland with legend
* Show the change dynamics in a bar plot
* Show the long term rainfall pattern
* Reflect on the major classes in 2000 and 2020 and the change observed

Submit these results in a one page short report.

# 11. References and tutorials<a name="reference"></a>
There are multiple tutorials and guides available now online on working with GEE. Below is a list of useful resources on GEE.
<br>
* [geemap tutorials by Prof. Quisheng Wu](https://geemap.org/tutorials/)
* [GEE developer's guide](https://developers.google.com/earth-engine/)
* [GEE API Docs](https://developers.google.com/earth-engine/apidocs)